In [32]:
# Necessary imports 
import numpy as np 
import pandas as pd 
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [33]:
df=pd.read_csv('DSL-StrongPasswordData.csv')

In [34]:
subjects = df['subject'].unique()

In [35]:
subjects_to_int = {subject: i  for i, subject in enumerate(subjects)}
int_to_subjects = {i: subject for i, subject in enumerate(subjects)}

In [36]:
df = df.replace(subjects_to_int)

C:\Users\skhim\AppData\Local\Temp\ipykernel_22164\1725018810.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(subjects_to_int)


In [37]:
df.head()

,subject,sessionIndex,rep,H.period,DD.period.t,UD.period.t,H.t,DD.t.i,UD.t.i,H.i,...,H.a,DD.a.n,UD.a.n,H.n,DD.n.l,UD.n.l,H.l,DD.l.Return,UD.l.Return,H.Return
0,0,1,1,0.1491,0.3979,0.2488,0.1069,0.1674,0.0605,0.1169,...,0.1349,0.1484,0.0135,0.0932,0.3515,0.2583,0.1338,0.3509,0.2171,0.0742
1,0,1,2,0.1111,0.3451,0.2340,0.0694,0.1283,0.0589,0.0908,...,0.1412,0.2558,0.1146,0.1146,0.2642,0.1496,0.0839,0.2756,0.1917,0.0747
2,0,1,3,0.1328,0.2072,0.0744,0.0731,0.1291,0.0560,0.0821,...,0.1621,0.2332,0.0711,0.1172,0.2705,0.1533,0.1085,0.2847,0.1762,0.0945
3,0,1,4,0.1291,0.2515,0.1224,0.1059,0.2495,0.1436,0.1040,...,0.1457,0.1629,0.0172,0.0866,0.2341,0.1475,0.0845,0.3232,0.2387,0.0813
4,0,1,5,0.1249,0.2317,0.1068,0.0895,0.1676,0.0781,0.0903,...,0.1312,0.1582,0.0270,0.0884,0.2517,0.1633,0.0903,0.2517,0.1614,0.0818


In [38]:
data_raw = df.values
print(data_raw)
data_raw.shape

[[ 0.      1.      1.     ...  0.3509  0.2171  0.0742]
 [ 0.      1.      2.     ...  0.2756  0.1917  0.0747]
 [ 0.      1.      3.     ...  0.2847  0.1762  0.0945]
 ...
 [50.      8.     48.     ...  0.2017  0.0983  0.0905]
 [50.      8.     49.     ...  0.1917  0.0938  0.0931]
 [50.      8.     50.     ...  0.1993  0.1186  0.1018]]


(20400, 34)

In [39]:
data=data_raw[ :, 2:]
labels_raw = df['subject'].values
labels= labels_raw.reshape(labels_raw.shape[0],1)
data=np.hstack([data, labels])
print(data)

[[ 1.      0.1491  0.3979 ...  0.2171  0.0742  0.    ]
 [ 2.      0.1111  0.3451 ...  0.1917  0.0747  0.    ]
 [ 3.      0.1328  0.2072 ...  0.1762  0.0945  0.    ]
 ...
 [48.      0.0939  0.1189 ...  0.0983  0.0905 50.    ]
 [49.      0.0923  0.1294 ...  0.0938  0.0931 50.    ]
 [50.      0.0596  0.131  ...  0.1186  0.1018 50.    ]]


In [40]:
np.random.shuffle(data)
x=data[ : , :-1]
y=data[:,-1]
x.shape

(20400, 32)

In [41]:
print(x)

[[50.      0.105   0.2389 ...  0.2028  0.0975  0.0989]
 [25.      0.0808  0.2581 ...  0.581   0.4722  0.0847]
 [34.      0.0824  0.1174 ...  0.1828  0.1049  0.0887]
 ...
 [15.      0.0944  0.175  ...  0.2702  0.1565  0.0791]
 [ 3.      0.0895  0.1084 ...  0.1741  0.0999  0.0557]
 [ 1.      0.0973  0.5608 ...  2.0737  1.9843  0.1472]]


In [42]:
print(y)

[46. 20. 10. ... 44. 10. 34.]


In [43]:
# scaler = MinMaxScaler()
# x = scaler.fit_transform(x)

In [44]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=0)

In [45]:
print(X_train)
print(y_train)

[[30.      0.0934  0.2972 ...  0.1647  0.0885  0.0979]
 [ 2.      0.1     0.3365 ...  0.2784  0.1658  0.0791]
 [46.      0.1148  0.3601 ...  0.3998  0.2936  0.1093]
 ...
 [24.      0.08    0.1871 ...  0.3061  0.1891  0.0847]
 [14.      0.0845  0.124  ...  0.2725  0.1569  0.0855]
 [28.      0.0684  0.3306 ...  0.5739  0.5055  0.0795]]
[41. 43.  1. ... 13. 13. 14.]


In [46]:


# Create an XGBoost classifier
model = xgb.XGBClassifier(objective='multi:softmax', num_class=2, random_state=0)

# Train the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Print classification report
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.9259803921568628
Classification Report:
               precision    recall  f1-score   support

         0.0       0.77      0.87      0.82        39
         1.0       0.92      0.98      0.95        50
         2.0       0.79      0.77      0.78        30
         3.0       0.95      0.92      0.93        38
         4.0       0.80      0.82      0.81        34
         5.0       0.87      0.79      0.83        34
         6.0       0.95      0.95      0.95        41
         7.0       0.95      0.90      0.93        42
         8.0       0.91      0.93      0.92        43
         9.0       0.98      0.93      0.96        46
        10.0       0.87      0.89      0.88        38
        11.0       0.97      1.00      0.99        39
        12.0       0.96      0.98      0.97        44
        13.0       0.94      0.94      0.94        34
        14.0       1.00      0.95      0.98        44
        15.0       0.80      0.96      0.87        25
        16.0       0.86     

In [47]:
# Testing for some data
# V
x = [[0.081152201,0.045574188,0.292444229,0.338018417,0.074759722,0.120056152,0.194815874,0.086019278,1.517424822,1.603444099,0.062575817,0.551804543,0.61438036,0,-1709202964,-1709202964,0.17720294,1709202965,1709202965,0.104067564,0.124180317,0.228247881,0.101912022,0.059402227,0.161314249,0.112463236,0.170367718,0.282830954,0,2
]]
model.predict(x)

ValueError: Feature shape mismatch, expected: 32, got 30

In [ ]:
# S
model.predict([[0.063287735,0.084783792,0.381062508,0.4658463,0.038760185,0.202909231,0.241669416,0.066504478,0.471563578,0.538068056,0.035029888,0.732656717,0.767686605,-1709201385,0.929215431,-1709201384,0.089103222,0.209048271,0.298151493,0.07996726,0.295785666,0.375752926,0.1405406,0.208619356,0.349159956,0.036897659,0.156013489,0.192911148,0,0
]])

array([1])

In [ ]:
# G
model.predict([[0.087309837,0.085243225,0.304038048,0.389281273,0.097291708,0.2170825,0.314374208,0.10254693,0.166110277,0.268657207,0.078619003,0.940005064,1.018624067,0.102619648,0.583746433,0.686366081,0.112853527,0.439263821,0.552117348,0.163859129,0.061246634,0.225105762,0.101477146,0.194048643,0.295525789,0.101518631,0.224496841,0.326015472,0,0
]])

array([0])